In [43]:
import numpy as np
%matplotlib tk
import matplotlib.pyplot as plt
import pickle
from sklearn import cluster
from sklearn import metrics

from sympy.solvers import solve
import sympy as sym

from scipy import optimize

In [46]:
class VelocityPlotter():
    def __init__(self):
        personNames = ['person1','person2','person3', 'person4','person5','person6']
        colors = ['red', 'green', 'orange', 'cyan', 'magenta', 'black']

        picklesToLoad = ['person1.pickle', 'person2.pickle','person3.pickle', 'person4.pickle', 'person5.pickle', 'person6.pickle']
        startTimes = [[26.8, 382, 483.3], #person1
            [3.6, 352, 446.75], # person2
            [9.5, 378, 481.25], # person3
            [35.1, 436, 543], #person4
            [10.8, 387, 500], # person5
            [12.1, 364, 458.25], #person6
            ]

        self.x, self.y = sym.symbols('x y')
        self.k1, self.k2 = sym.symbols('k1 k2')

        self.eq_y = None
        self.eq_x = None

        dataSets = []
        for pickleToLoad in picklesToLoad:
            data = pickle.load(open(pickleToLoad, 'rb'))
            dataSets.append(data)

        newDataset = [[[] for _ in range(4)] for _ in range(len(dataSets))]
        for i, dataSet in enumerate(dataSets):
            x_log, distance_to_spot_log, time_log_angle, time_log_filter, spot_v_log, time_log_spot = dataSet
            for data in zip(x_log, distance_to_spot_log, time_log_angle, time_log_filter):
                if startTimes[i][1] > data[3] > startTimes[i][0]:
                    newDataset[i][0].append(data[0])
                    newDataset[i][1].append(data[1])
                    newDataset[i][2].append(data[2])
                    newDataset[i][3].append(data[3])
        self.all_distance = []
        self.all_vel = []
        self.all_angles = []

        fig2, ax2 = plt.subplots(1)

        for j, dataSet in enumerate(newDataset):
            x_log, distance_to_spot_log, time_log_angle, time_log_filter = dataSet

            #fig, ax = plt.subplots(1)
            for i in range(len(time_log_angle)):
                time_log_angle[i] = (time_log_angle[i]-np.pi) * (180/np.pi)

            person_velocity = []
            for i in range(len(x_log)):
                person_velocity.append( np.linalg.norm( np.array(x_log[i])[3:5] ) )

            bool_array = np.array(person_velocity) > 0.25
            person_velocity = list(np.array(person_velocity)[bool_array == True])
            distance_to_spot_log = list(np.array(distance_to_spot_log)[bool_array == True])
            time_log_angle = list(np.array(time_log_angle)[bool_array == True])
            time_log_filter = list(np.array(time_log_filter)[bool_array == True])

            #print(len(distance_to_spot_log))
            #print(len(time_log_angle))
            #print(len(time_log_filter))

            self.all_distance = self.all_distance + distance_to_spot_log
            self.all_vel = self.all_vel  + person_velocity
            self.all_angles = self.all_angles + time_log_angle

            filter = np.array(distance_to_spot_log) < 3.5
            coefs = np.polyfit(np.array(person_velocity)[filter == True], np.array(distance_to_spot_log)[filter == True], 1)

            # X = np.arange(0.1, 1.4, 0.1)
            # ax.plot(X, np.polyval(coefs, X), color="black")
            # ax.plot(np.array(person_velocity)[filter == True], np.array(distance_to_spot_log)[filter == True], 'o', label=personNames[j], color=colors[j])
            # ax.set_ylabel('Distance to Spot [m]')
            # ax.set_xlabel('Speed [m/s]')
            # ax.legend()
            # ax.set_ylim([0.5, 3.5])

            ax2.plot(np.array(person_velocity)[filter == True], np.array(distance_to_spot_log)[filter == True], 'o', markersize=1, label=personNames[j], color=colors[j])
            ax2.set_ylabel('Distance to Spot [m]')
            ax2.set_xlabel('Speed [m/s]')
            ax2.legend()
            ax2.set_ylim([0.5, 3.5])
            ax2.set_xlim([0.1, 1.5])
            #print("dist:" ,personNames[j], np.median(distance_to_spot_log))
            #print("angle:" ,personNames[j], np.median(time_log_angle))


        #fig, ax3 = plt.subplots(1)
        self.X = np.arange(-2, 5, 0.5)
        self.Y = np.arange(0, 10, 0.5)

        filter = np.array(self.all_distance) < 3.5
        self.coefs = np.polyfit(np.array(self.all_vel)[filter == True], np.array(self.all_distance)[filter == True], 1)
        #self.coefs = np.array([1, 0])

        self.filtered = (np.array(self.all_vel)[filter == True], np.array(self.all_distance)[filter == True])
        ax2.plot(self.X, np.polyval(self.coefs, self.X), color="black")

        fig2, ax5 = plt.subplots(1)

        def point_on_curve(x0, y0):
            x_min = sym.Symbol('x')
            mean = np.polyval(self.coefs, x_min)
            the_diff = sym.diff( sym.sqrt( (x_min - x0)**2 + (mean - y0)**2 ), x_min )
            return_var = solve(the_diff, x_min)
            return float(sym.re(return_var[0])), float(sym.re(np.polyval(self.coefs, return_var[0])))

        def equation(x):
            x_min, y_min = point_on_curve(x[0], x[1])

            distance = sym.sqrt( (x_min - x[0])**2 + (y_min - x[1])**2 )#

            x1 = 1.4
            y1 = np.polyval(self.coefs, x1)

            arc_length = sym.sqrt((x1 - x_min)**2 + (y1 - y_min)**2)
            return float(sym.re(distance + arc_length))

        def func3d(x0, y0):
            x2, y2 = point_on_curve(x0, y0)
            dist = sym.sqrt( (x2 - x0)**2 + (y2 - y0)**2 )

            x1 = 1.4
            y1 = np.polyval(self.coefs, x1)

            arc_length = np.linalg.norm([x1 - x2,
                                        y1 - y2])

            if float(sym.re(dist + arc_length)) < 0.2:
                return 0.2

            return float(sym.re(dist + arc_length))

        def point_on_line(x0, y0):
            a,b = self.coefs[0], self.coefs[1]

            y = y0 - b
            vector = np.array([1, a])
            the_length = np.dot(np.array([x0, y]), vector) / np.dot(vector, vector)
            point = (the_length * vector)
            return point[0], point[1] + b

        def func3dLinear(x0, y0):
            x1, y1 = point_on_line(x0, y0)

            kp_dist = 0.2
            kp_vel = 0.4

            #return sym.sqrt((x0500 - x1)**2 + (y0 - y1)**2)
            #return sym.sqrt( (1.4 - x1)**2 + (np.polyval(self.coefs, 1.4) - y1)**2 ) + sym.sqrt((x0 - x1)**2 + (y0 - y1)**2)**2
            return self.k1*((np.polyval(self.coefs, x0) - y0)**2) + self.k2*((1.4 - x0)**2)

        def quiver(x0, y0):
            if self.eq_x == None:
                eq = func3dLinear(self.x, self.y)
                self.eq_x = eq.diff(self.x)
                self.eq_y = eq.diff(self.y)

            return (float(self.eq_x.subs(self.x, x0).subs(self.y, y0).subs(self.k1, 1).subs(self.k2, 1)),
             float(self.eq_y.subs(self.x, x0).subs(self.y, y0).subs(self.k1, 1).subs(self.k2, 1)))

        self.grads = np.zeros((len(self.X), len(self.Y), 2))
        for ind_x, x in enumerate(self.X):
            for ind_y, y in enumerate(self.Y):
                values = quiver(x, y)
                self.grads[ind_x, ind_y, :] = np.asfarray(values)
                ax5.quiver(x, y, -values[0], -values[1])

        #func3d_vectorized = np.vectorize(func3dLinear)
        #self.Z = np.zeros((len(self.Y),len(self.X)))
        #for x_ind, x_val in enumerate(self.X):
        #    for y_ind, y_val in enumerate(self.Y):
        #        self.Z[y_ind, x_ind] = func3dLinear(x_val, y_val)
    
        #X, Y = np.meshgrid(self.X, self.Y)
        #self.Z = np.asfarray(func3d_vectorized(Y, X))



letsgoo = VelocityPlotter()

In [47]:
fig, ax3 = plt.subplots(1)
cm = plt.cm.get_cmap('viridis')
#ax3.contourf(letsgoo.X, letsgoo.Y, letsgoo.Z, 50)
#ax3.plot(letsgoo.X, np.polyval(letsgoo.coefs, letsgoo.X), 'r-')
#ax3.set_xlim([0,5])
#ax3.set_ylim([0,5])
#letsgoo.coefs

print([sym.simplify(letsgoo.eq_x), sym.simplify(letsgoo.eq_y)])

AttributeError: 'VelocityPlotter' object has no attribute 'Z'

In [ ]:
fig, ax3 = plt.subplots(1)
cm = plt.cm.get_cmap('viridis')
ax3.plot(letsgoo.X, np.polyval(letsgoo.coefs, letsgoo.X), 'r-')

x1, y1 = 1, 2

ax3.plot(x1, y1, 'ro')


def point_on_line(x0, y0):
    a,b = letsgoo.coefs[0], letsgoo.coefs[1]
    y = y0 - b
    vector = np.array([1, a])
    the_length = np.dot(np.array([x0, y]), np.transpose(vector)) / np.dot(np.transpose(vector), vector)
    point = (the_length * vector) + np.array([0, b])
    return point


ax3.plot(point_on_line(x1,y1)[0], point_on_line(x1,y1)[1], 'ro')

print(sym.sqrt( (1.4 - point_on_line(x1,y1)[0])**2 + (np.polyval(letsgoo.coefs, 1.4) - point_on_line(x1,y1)[1])**2 ))


0.525930084844368


In [ ]:
fig, ax3 = plt.subplots(1)
cm = plt.cm.get_cmap('viridis')
ax3.contourf(letsgoo.X, letsgoo.Y, letsgoo.Z)

x_i = 1
y_i = 1
Last_cost = letsgoo.Z[y_i, x_i]
iteration = 0
while True: #not (1.4 + 0.01 > letsgoo.X[x_i] > 1.4 - 0.01):
    iteration += 1
    if iteration > 50:
        break
    new_x = x_i
    new_y = y_i
    for x in range(-1, 2):
        for y in range(-1, 2):
            if letsgoo.Z[y_i + y, x_i + x] < Last_cost:
                new_x = x_i + x
                new_y = y_i + y
                Last_cost = letsgoo.Z[y_i + y, x_i + x]

    x_i = new_x
    y_i = new_y
    
    ax3.plot(letsgoo.X[x_i], letsgoo.Y[y_i], 'o')

In [ ]:
vale = []
for i in range(1, 15):
    polynomial = np.polyfit(letsgoo.filtered[0], letsgoo.filtered[1], i, full=True)
    vale.append(polynomial[1] / len(letsgoo.filtered[0]))

    # r-squared
    p = np.poly1d(polynomial[0])
    # fit values, and mean
    yhat = p(letsgoo.filtered[0])                         # or [p(z) for z in x]
    ybar = np.sum(letsgoo.filtered[1])/len(letsgoo.filtered[1])          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((letsgoo.filtered[1] - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    print("rSquared: ", ssreg/sstot)

vale

rSquared:  0.35910237303077575
rSquared:  0.3780233587026233
rSquared:  0.4094850453367208
rSquared:  0.4145768119160963
rSquared:  0.415720663593014
rSquared:  0.41595788434227726
rSquared:  0.4164028486118921
rSquared:  0.4164061203975021
rSquared:  0.4214773482509907
rSquared:  0.4230772684668947
rSquared:  0.4230772699172622
rSquared:  0.4259557357191395
rSquared:  0.4275705445138496
rSquared:  0.4407208059074988


[array([0.12173258]),
 array([0.11813871]),
 array([0.11216285]),
 array([0.11119572]),
 array([0.11097846]),
 array([0.1109334]),
 array([0.11084888]),
 array([0.11084826]),
 array([0.10988503]),
 array([0.10958114]),
 array([0.10958114]),
 array([0.10903439]),
 array([0.10872767]),
 array([0.10622979])]

In [ ]:
plt.plot(range(1, len(vale) + 1),vale)

In [ ]:
fig = plt.figure()
ax3 = fig.gca(projection='3d')

X, Y = np.meshgrid(letsgoo.X, letsgoo.Y)
Z = letsgoo.Z

ax3.plot_surface(X, Y, Z, cmap='jet')

ax3.set_ylabel('Distance to spot [m]')
ax3.set_xlabel('Speed [m/s]')
ax3.set_zlabel('Cost')

/tmp/ipykernel_18162/571970758.py:2: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax3 = fig.gca(projection='3d')


Text(0.5, 0, 'Cost')

In [ ]:
import pickle

pickle.dump((letsgoo.X, letsgoo.Y, letsgoo.grads), open("gradients.pickle", "wb"))

In [ ]:
fig2, ax5 = plt.subplots(1)
X, Y, grads = pickle.load(open("gradients.pickle", 'rb'))
for ind_x, x in enumerate(X):
    for ind_y, y in enumerate(Y):
        ax5.quiver(x, y, -grads[ind_x, ind_y, 0], -grads[ind_x, ind_y, 1])